In [16]:
import polars as pl

In [17]:
lf = pl.read_csv("merged_output.csv", has_header=True, ignore_errors=True).lazy()


In [18]:
lf = lf.rename({
    "Grouping": "GroupingCol",
    "BroadcasterChannelConfigurationVideoDontUseDefaultMixerResolution": "BroadCasterChannelShortened",
    "ClipMainUserAreaAcquisitionMetadataCameraUnitMetadataGammaCaptureGamma": "ClipMainUserAreaAcquisitionMetadataShortened",
    # "ClipMainUserAreaAcquisitionMetadataCameraUnitMetadataGammaCaptureGammaValue": "ClipMainUserAreaAcquisitionMetadataShortenedValue",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskColorAmount": "PantryCircularGradientBasedShortened",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskDepthFeather": "PCGC_CRM_DepthFeather",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskDepthMax": "PCGC_CRM_DepthMax",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskDepthMin": "PCGC_CRM_DepthMin",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskLumFeather": "PCGC_CRM_LumFeather",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskLumMax": "PCGC_CRM_LumMax",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskLumMin": "PCGC_CRM_LumMin",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskPointModels": "PCGC_CRM_PointModels",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskType": "PCGC_CRM_Type",
    "PantryCircularGradientBasedCorrectionsCorrectionRangeMaskVersion": "PCGC_CRM_Version",
    "PantryCircularGradientBasedCorrectionsLocalBlacks2012": "PCGC_LocalBlacks2012",
    "PantryCircularGradientBasedCorrectionsLocalDehaze": "PCGC_LocalDehaze",
    "PantryCircularGradientBasedCorrectionsLocalHue": "PCGC_LocalHue",
    "PantryCircularGradientBasedCorrectionsLocalTexture": "PCGC_LocalTexture",
    "PantryCircularGradientBasedCorrectionsLocalWhites2012": "PCGC_LocalWhites2012"
})

In [20]:
# Step 3: Collect into memory (must collect before writing)
df = lf.collect()

# Step 4: Write the DataFrame back to a new CSV
df.write_csv("renamed_output.csv")

# Remove duplicate sql table stuff

In [ ]:
import re

def deduplicate_sql_columns(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as f:
        sql = f.read()

    # Find the CREATE TABLE block with contents inside ()
    match = re.search(r'CREATE\s+TABLE\s+\w+\s*\((.*?)(\);)', sql, re.IGNORECASE | re.DOTALL)
    if not match:
        print("CREATE TABLE block not found.")
        return

    columns_block = match.group(1)
    closing = match.group(2)

    # Split into lines and deduplicate by column name
    seen = set()
    deduped_lines = []
    for line in columns_block.splitlines():
        stripped = line.strip()
        if not stripped:
            continue

        col_match = re.match(r'^(\w+)\s+', stripped)
        if col_match:
            col_name = col_match.group(1)
            if col_name not in seen:
                seen.add(col_name)
                deduped_lines.append(line.rstrip(", \n") + ",")
            else:
                print(f"Duplicate column removed: {col_name}")
        else:
            deduped_lines.append(line)  # fallback for malformed lines

    # Remove trailing comma from the last column
    if deduped_lines:
        deduped_lines[-1] = deduped_lines[-1].rstrip(",")  # last line no comma

    # Rebuild and write output
    start = sql[:match.start(1)]
    end = closing + sql[match.end(2):]
    deduped_sql = start + "\n   " + "\n   ".join(deduped_lines) + "\n" + end

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(deduped_sql)

    print(f"Deduplicated SQL written to: {output_path}")

# Usage
deduplicate_sql_columns("table_creation.sql", "table_creation_dedupe.sql")


Deduplicated SQL written to: table_creation_dedupe.sql


In [ ]:
import re

def fix_column_commas(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    fixed_lines = []
    buffer = []

    for line in lines:
        # Trim newline for easier processing
        stripped = line.rstrip("\n")

        # If line starts with a comma, move it to end of previous line
        if re.match(r'^\s*,', stripped) and buffer:
            # Remove leading comma
            stripped = re.sub(r'^\s*,\s*', '', stripped)
            # Add comma to previous buffered line
            buffer[-1] = buffer[-1].rstrip(',') + ','
        
        buffer.append(stripped)

    # Add commas to all but last column line inside CREATE TABLE
    in_create = False
    for i, line in enumerate(buffer):
        if re.search(r'CREATE\s+TABLE\s+\w+\s*\(', line, re.IGNORECASE):
            in_create = True
            fixed_lines.append(line)
            continue

        if in_create:
            if line.strip().startswith(")"):
                in_create = False
                fixed_lines.append(line)
                continue

            # For lines inside CREATE TABLE block, ensure comma except last before ")"
            next_line = buffer[i + 1] if i + 1 < len(buffer) else ""
            if not line.rstrip().endswith(",") and not next_line.strip().startswith(")"):
                line += ","

        fixed_lines.append(line)

    # Write to output
    with open(output_path, "w", encoding="utf-8") as f:
        for line in fixed_lines:
            f.write(line + "\n")

    print(f"Fixed commas written to: {output_path}")

# Example usage:
fix_column_commas("table_creation_dedupe.sql", "fixed_commas_table.sql")


Fixed commas written to: fixed_commas_table.sql
